#### Notebook to prepair raw data  
date: 2023

#### Step 0. Imports


In [2]:
import pandas as pd  
import numpy as np  
import os

In [3]:
os.chdir(r"C:\Users\Administrateur\OneDrive\Documents\GitHub\reparatorAI")

#### Step1. Functions  

In [60]:
def first_clean_df(df):
    df_ok = df.copy()
    df_ok['brand']=['-'.join(str(my_brand).upper().strip().split(' ')[0:1]) for my_brand in df_ok.brand]
    df_ok['product_category']=[str(my_val).upper().strip() for my_val in df_ok.product_category]
    my_brand_low_freq = df_ok.brand.value_counts()[df_ok.brand.value_counts() < 5].index.tolist()
    df_ok = df_ok[df_ok.brand.isin(my_brand_low_freq) == False]
    df_ok['repair_barrier_if_end_of_life'] = df_ok['repair_barrier_if_end_of_life'].fillna('Unspecified')
    return df_ok

#### Step2. Datasets  

In [14]:
raw=pd.read_csv('./data/OpenRepairData_v0.3_aggregate_202309.csv', encoding='latin', low_memory=False)

In [15]:
raw.shape

(133426, 14)

In [16]:
raw.columns

Index(['id', 'data_provider', 'country', 'partner_product_category',
       'product_category', 'product_category_id', 'brand',
       'year_of_manufacture', 'product_age', 'repair_status',
       'repair_barrier_if_end_of_life', 'group_identifier', 'event_date',
       'problem'],
      dtype='object')

In [10]:
old=pd.read_csv('./data/OpenRepairData_v0.3_aggregate_202303.csv', index_col=0, encoding='latin', low_memory=False)

In [11]:
old.shape

(103076, 21)

In [13]:
old.columns

Index(['id', 'data_provider', 'country', 'partner_product_category',
       'product_category', 'product_category_id', 'brand',
       'year_of_manufacture', 'product_age', 'repair_status',
       'repair_barrier_if_end_of_life', 'group_identifier', 'event_date',
       'problem', 'product_category_new', 'brand_ok', 'TopCategory',
       'problem_class', 'TopCategory_FR', 'product_category_FR',
       'problem_class_main'],
      dtype='object')

In [18]:
selectObjectList_UK=['POWER TOOL', 'TOY', 'HAIR DRYER', 'DECORATIVE OR SAFETY LIGHTS', 'LAMP',
 'PORTABLE RADIO', 'HANDHELD ENTERTAINMENT DEVICE', 'FOOD PROCESSOR', 'SMALL HOME ELECTRICAL',
 'HAIR & BEAUTY ITEM', 'MISC', 'SEWING MACHINE', 'WATCH/CLOCK', 'HI-FI SEPARATES',
 'DESKTOP COMPUTER', 'BATTERY/CHARGER/ADAPTER', 'SMALL KITCHEN ITEM', 'VACUUM',
 'TV AND GAMING-RELATED ACCESSORIES', 'COFFEE MAKER', 'KETTLE', 'IRON',
 'DIGITAL COMPACT CAMERA', 'PRINTER/SCANNER', 'LAPTOP', 'HI-FI INTEGRATED',
 'PAPER SHREDDER', 'TOASTER', 'FLAT SCREEN', 'MOBILE', 'TABLET', 'DSLR/VIDEO CAMERA',
 'HEADPHONES', 'LARGE HOME ELECTRICAL', 'MUSICAL INSTRUMENT', 'PROJECTOR',
 'PC ACCESSORY', 'AIRCON/DEHUMIDIFIER', 'FAN', 'GAMES CONSOLE']

selectObjectList_FR=['Outil Bricolage', 'Jouet', 'Sèche cheveux', 'Luminaires et guirlandes déco',
 'Lampe', 'Radio portable', 'Appareil de divertissement portable', 'Robot de cuisine', 'Petit électroménager de maison',
 'équipement pour cheveux & beauté', 'Divers', 'Machine à coudre', 'Montre / Réveil', 'Composants HI-Fi', 'Ordinateur de Bureau',
 'Batterie / chargeur / adaptateur', 'Petit électroménager de cuisine', 'Aspirateur', 'accessoire TV et jeux videos', 'Machine à café',
 'Bouilloire', 'Fer à repasser', 'Appareil photo numérique', 'Imprimante / scanner', 'Ordinateur portable', 'Hi-Fi', 'Broyeuse à papier',
 'Grille pains', 'Ecran plat', 'Téléphone portable', 'Tablette', 'Camescope', 'Ecouteurs', 'Gros électroménager',
 'Instrument de musique', 'Vidéo projecteur', 'Accessoire PC', 'Climatiseur / déshumidificateur', 'Ventilateur', 'Console de jeux vidéo']
selectObjectList_FR=[my_str.upper() for my_str in selectObjectList_FR]

In [19]:
topCategory_uk=['BATHROOM', 'ELECTRONICS', 'HOME', 'IMAGE', 'KITCHEN', 'OFFICE', 'OTHER', 'SOUND']
topCategory_fr=['SALLE DE BAIN','ELECTRONIQUE','MAISON','IMAGE','CUISINE','BUREAU','AUTRES','SON']

#### Step3. final treatment and saving  
New columns to create : 'product_category_new', 'brand_ok', 'TopCategory',
       'problem_class', 'TopCategory_FR', 'product_category_FR',
       'problem_class_main'

##### step3.1: create dict objects to map the final dataset   


In [49]:
dict_topCategory=old.groupby(by=['TopCategory','product_category_new']).count().iloc[:,0].index.tolist()
dict_topCategory_all={}
for obj in dict_topCategory:
    dict_topCategory_all[obj[1]]=obj[0]

In [39]:
dict_topCategory_ENtoFR={}
for i in range(len(topCategory_uk)):
    dict_topCategory_ENtoFR[topCategory_uk[i]]=topCategory_fr[i]

In [41]:
dict_objList_ENtoFR={}
for i in range(len(selectObjectList_UK)):
    dict_objList_ENtoFR[selectObjectList_UK[i]]=selectObjectList_FR[i]

##### step3.2: map clean dataset with dict objects

In [61]:
new=first_clean_df(raw)
new["TopCategory"]=new["product_category"].map(dict_topCategory_all)
new["TopCategory_FR"]=new["TopCategory"].map(dict_topCategory_ENtoFR)
new["product_category_FR"]=new["product_category"].map(dict_objList_ENtoFR)

In [62]:
new.head()

,id,data_provider,country,partner_product_category,product_category,product_category_id,brand,year_of_manufacture,product_age,repair_status,repair_barrier_if_end_of_life,group_identifier,event_date,problem,TopCategory,TopCategory_FR,product_category_FR
0,anstiftung_2746,anstiftung,DEU,Computer ~ Drucker,PRINTER/SCANNER,25,UNKNOWN,NaN,NaN,Fixed,Unspecified,5073,2012-06-20,Funktionierte nicht mehr. Fehler und LÃ¶sung n...,OFFICE,BUREAU,IMPRIMANTE / SCANNER
1,anstiftung_2747,anstiftung,DEU,Unterhaltungselektronik ~ KopfhÃ¶rer,HEADPHONES,11,UNKNOWN,NaN,NaN,Fixed,Unspecified,5073,2012-06-20,Funktionierte nicht mehr. Fehler und LÃ¶sung n...,SOUND,SON,ECOUTEURS
2,anstiftung_2750,anstiftung,DEU,Computer ~ Laptop,LAPTOP,16,UNKNOWN,NaN,NaN,Repairable,Unspecified,5073,2012-06-20,Wurde schnell heiÃ. Der LÃ¼fter war sehr vers...,ELECTRONICS,ELECTRONIQUE,ORDINATEUR PORTABLE
3,anstiftung_2749,anstiftung,DEU,Elektro divers ~ NÃ¤hmaschine,SEWING MACHINE,27,UNKNOWN,NaN,NaN,Repairable,Unspecified,5073,2012-06-20,Funktionierte nicht mehr. Fehler und LÃ¶sung n...,HOME,MAISON,MACHINE À COUDRE
4,anstiftung_2742,anstiftung,DEU,HaushaltsgerÃ¤te ~ Spielzeug,TOY,32,UNKNOWN,NaN,NaN,Fixed,Unspecified,5073,2012-09-19,Die Beine der Puppe waren ab. Sie waren mit FÃ...,OTHER,AUTRES,JOUET


In [63]:
new.shape

(123217, 17)